# 1. Đặt vấn đề


Trong lập trình C hay C++ chúng ta hay làm việc với con trỏ. Trong một chương trình lớn chúng ta phải quản lý một số 
lượng lớn con trỏ khác nhau, sau khi dùng xong chúng ta luôn phải giải phóng vùng nhớ con trỏ đó đang chỉ tới.

In [1]:
{
int* p = new int [10];
delete[] p; // giải phóng vùng nhớ  
}


Hoặc trong quá trình sử dụng ta có thể vô tình giải phóng con trỏ đó 2 lần. Hành động này dẫn đến lúc runtime bị **Undefined Behavior**.

In [2]:
// Các bạn mà xóa dấu commnet ở dưới rồi chạy là jupyter notebook error -> reset đấy nha =)))
{
int* p = new int [10];
    {
        delete[] p;
    }
    
//delete[] p;  // Undefined Behavior
}

>Sau khi chúng ta cấp phát bộ nhớ cho dùng cho chương trình thì ta cần phải xóa con trỏ đó đi để không gây lãng phí memory, leak memory. Với chương trình nhỏ việc quản lý các con trỏ thủ công tương đối dễ dàng, Tuy nhiên với các chương trình có mã nguồn lớn việc quản lý con trỏ thô này khá là khó khăn. Các anh em developer C thầm nghĩ đó là chuyện cơm bữa rồi, tuy nhiên với ngôn ngữ C++, những feature mới người ta có sáng tạo ra con trỏ thông minh - **Smart Poiter**. Thứ giúp ta quản lý con trỏ một cách tự động, ta không cần phải lo nghĩ đến việc phải giải phóng bộ nhớ mỗi khi dùng xong con trỏ. Vậy ta hãy đi tìm hiểu xem nó thực sự có **Smart** không thật nha. 

# 2. Cơ chế liên quan giải phóng bộ nhớ

Đây mình lấy ví dụ cho thấy nếu con trỏ không xóa thì việc gì sẽ xảy ra. 

In [3]:
#include <iostream>
class Memory {
public:
Memory() {
    std::cout << "Cấp phát vùng nhớ\n";
}

~Memory() {
    std::cout << "Thu hồi vùng nhớ\n";
    
}
};


{
    Memory* pM = new Memory();
}

std::cout << "---------------------\n";
{
   Memory m;
}


std::cout << "---------------------\n";
{
   Memory* pM = new Memory();
    delete pM;
    //pM->~Memory();
}


Cấp phát vùng nhớ
---------------------
Cấp phát vùng nhớ
Thu hồi vùng nhớ
---------------------
Cấp phát vùng nhớ
Thu hồi vùng nhớ


Kết quả run xong các bạn sẽ thấy thắc mắc tại sao thằng *pM không gọi được tới hàm hủy. Vì **Memory m** được khởi tạo tại stack nên khi out scope tự động gọi hàm Destructor. Còn **Memory \*pM** giá trị của con trỏ *pM được khởi tạo tại **Stack**, còn dữ liệu được trỏ tới đối tượng **Memory** được khởi tạo tại **Heap**. Với dự liệu được khởi tạo trong **Heap** khác với dữ liệu khởi tạo tại **Stack** là chúng ta phải quản lý chúng một cách thủ công. Nên là khi con trỏ **\*pM** out scope thì tuy giá trị con trỏ được phóng nhưng dữ liệu con trỏ tới tại **Heap** chưa được giải phóng --> leak memory.


[Tham khảo thêm.](https://stackoverflow.com/questions/56588367/why-is-the-destructor-not-called-when-im-declaring-a-pointer-to-an-object)


# 3. Bí mật đằng sau Smart Poiter

Ta xem qua ví dụ code về **Smart pointer** nhé:


``` cpp
//Các bạn code trình biên dịch nào đó họăc c++ compiler online run cũng được. Vì code dưới không thể chạy được trên xeus-cling. :((((
#include <iostream>

class Memory {
public:
    Memory() {
        std::cout << "Memory Constructor\n";
    }

    ~Memory() {
        std::cout << "Memory Destructor\n";
    }
};

template <typename T>
class SmartPointer {
public:
    SmartPointer() {
        std::cout << "SmartPointer Constructor\n";
        t = new T();
    }

    ~SmartPointer() {
        std::cout << "SmartPointer Destructor\n";
        delete t;
    }

    SmartPointer(const SmartPointer& other) {
        std::cout << "SmartPointer Copy Constructor\n";
        t = new T(*other.t); // Sao chép đối tượng từ pointer
    }

    // Toán tử gán - ngăn gán sai cách
    SmartPointer& operator=(const SmartPointer& other) {
        if (this == &other) return *this; // Tránh tự gán

        delete t; // Giải phóng bộ nhớ hiện tại
        t = new T(*other.t); // Sao chép đối tượng
        return *this;
    }


    
private:
    T* t;
};
int main() {
 {
    SmartPoiter<Memory> pSmart; 
}
    return 0;
}
```

Các bạn đọc xong code ngẫm nghĩ chút rồi đoán xem ý tưởng của **Smart Poiter** là gì. Ý tưởng của nó thực ra là dựa trên ý tưởng là 1 class sẽ wrap(SmartPointer) là lại dữ liệu được cấp phát tại heap mà ta muốn quản lý. KỸ hơn là giá trị khởi tạo pSmart được khởi tạo trong **Stack** nên khi out scpoe thì pSmart này sẽ tự động gọi đến hàm Destructor của nó, rồi nó sẽ gọi tới hàm hàm delete. Từ đó giải phóng giữ liệu trên heap của dữ liệu **T** mà class SmartPointer quản lý.


> Riêng có 2 hàm Copy Constructor và Move Constructor là để cho mục đích tạo ra các loại **Smart Poiter** mà mình sẽ đề cập bên dưới.